In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
# tf.config.experimental.set_memory_growth()
tf.test.is_gpu_available()

True

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#         tf.config.experimental.set_memory_growth(gpu, True)
# tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3096)])
# # tf.config.experimental.set_memory_growth()

In [3]:
tf.__version__

'2.1.0'

## Load data

In [4]:
data = pd.read_csv('data/cards.csv')    
prices = pd.read_csv('data/prices.csv')

pd.options.display.max_columns = data.shape[1] #display option
# data.describe(include='all')

C:\Users\Przemyslaw\.conda\envs\mtg\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,9,12,16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
prices_no_duplicates = prices.drop_duplicates(subset=['uuid'])
# prices_no_duplicates.describe(include='all')

In [6]:
# prices.head()
# data.head()
data.columns

Index(['index', 'id', 'artist', 'asciiName', 'borderColor', 'colorIdentity',
       'colorIndicator', 'colors', 'convertedManaCost', 'duelDeck',
       'edhrecRank', 'faceConvertedManaCost', 'flavorName', 'flavorText',
       'frameEffect', 'frameEffects', 'frameVersion', 'hand', 'hasFoil',
       'hasNoDeckLimit', 'hasNonFoil', 'isAlternative', 'isArena', 'isBuyABox',
       'isDateStamped', 'isFullArt', 'isMtgo', 'isOnlineOnly', 'isOversized',
       'isPaper', 'isPromo', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'layout',
       'leadershipSkills', 'life', 'loyalty', 'manaCost', 'mcmId', 'mcmMetaId',
       'mtgArenaId', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'names',
       'number', 'originalText', 'originalType', 'otherFaceIds', 'power',
       'printings', 'purchaseUrls', 'rarity', 'scryfallId',
       'scryfallIllustrationId', 'scryfallOracleId', 'setCode', 'side',
       'subtypes', 'supertypes', 'tcgplayerProduc

In [7]:
data_merged = pd.merge(data, prices_no_duplicates, on='uuid', how='inner')

In [8]:
data_merged.drop_duplicates(subset='name', keep="last", inplace=True)

### Include selected features

In [9]:
# len(data_merged)
data_merged = data_merged[['name', 'text', 'price', 'convertedManaCost', 'rarity', 'power', 'life', 'colors']]
# print(data_merged.dtypes)

In [10]:
data_merged[data_merged.price > 100][['name', 'text', 'price', 'convertedManaCost']]

,name,text,price,convertedManaCost
5003,Ali from Cairo,Damage that would reduce your life total to le...,304.95,4.0
5006,Bazaar of Baghdad,"{T}: Draw two cards, then discard three cards.",1499.99,0.0
5012,City in a Bottle,Whenever one or more other nontoken permanents...,399.95,2.0
5021,Diamond Valley,"{T}, Sacrifice a creature: You gain life equal...",250.99,0.0
5022,Drop of Honey,"At the beginning of your upkeep, destroy the c...",419.96,1.0
...,...,...,...,...
34102,Ravages of War,Destroy all lands.,179.92,4.0
34182,Zodiac Dragon,When Zodiac Dragon is put into your graveyard ...,243.20,9.0
35944,Grim Tutor,Search your library for a card and put that ca...,231.87,3.0
39121,"Ugin, the Spirit Dragon","[+2]: Ugin, the Spirit Dragon deals 3 damage t...",138.21,8.0


### Zamiania typu na string dla tokenizera

In [11]:
data_merged['text'] = data_merged['text'].astype(str)

### Tokenizacja

In [12]:
# vocab_size = 1000 #cannot be too long for lstm layer
vocab_size = 2048
# max_length = 287 #95% cards will fit this text length
max_length = 196 #80%

# max_length = data_merged.text.map(lambda x: len(x)).max()
# max_length = int((max_length * 3) / 4)

# max_length = 100
trunc_type='post'
# padding_type='post'
padding_type = 'pre'

In [13]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(data_merged['text'])
sequences = tokenizer.texts_to_sequences(data_merged['text'])

In [14]:
tokenizer.word_index

{'<OOV>': 1,
 'the': 2,
 'creature': 3,
 'you': 4,
 'a': 5,
 'of': 6,
 '1': 7,
 'your': 8,
 'target': 9,
 'to': 10,
 'this': 11,
 'card': 12,
 'it': 13,
 'turn': 14,
 'control': 15,
 '2': 16,
 'battlefield': 17,
 'or': 18,
 'and': 19,
 'that': 20,
 'if': 21,
 'damage': 22,
 'end': 23,
 'may': 24,
 'on': 25,
 'put': 26,
 'with': 27,
 'each': 28,
 'player': 29,
 'until': 30,
 'enters': 31,
 'as': 32,
 'from': 33,
 'whenever': 34,
 'when': 35,
 'creatures': 36,
 't': 37,
 'cast': 38,
 'library': 39,
 'for': 40,
 'its': 41,
 'flying': 42,
 'cards': 43,
 'an': 44,
 'spell': 45,
 'hand': 46,
 'deals': 47,
 'counter': 48,
 'gets': 49,
 'at': 50,
 'graveyard': 51,
 '3': 52,
 'any': 53,
 'life': 54,
 'sacrifice': 55,
 "can't": 56,
 'cost': 57,
 'x': 58,
 'be': 59,
 'is': 60,
 'mana': 61,
 'draw': 62,
 'beginning': 63,
 'then': 64,
 'in': 65,
 'exile': 66,
 'all': 67,
 'two': 68,
 'into': 69,
 'b': 70,
 'their': 71,
 'opponent': 72,
 'return': 73,
 'land': 74,
 'r': 75,
 'gain': 76,
 'top': 77,


### Padding

In [15]:
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
# padded

### Attach converted mana cost

In [16]:
data_merged['convertedManaCost'].to_numpy().reshape(-1, 1)

array([[4.],
       [4.],
       [5.],
       ...,
       [5.],
       [2.],
       [5.]])

In [17]:
padded = np.append(padded, data_merged['convertedManaCost'].to_numpy().reshape(-1, 1), axis=1)
max_length = max_length + 1
# padded = data_merged['convertedManaCost'].to_numpy().reshape(-1, 1)
# max_length = 1

### get rarity dummies

In [18]:
padded[0].shape

(197,)

In [19]:
# data_merged.rarity
# pd.get_dummies(data_merged['rarity'])
padded = np.append(padded, pd.get_dummies(data_merged['rarity'], drop_first=True).to_numpy(), axis=1)
max_length = max_length + 3

### Power

In [20]:
data_merged['power'] = pd.to_numeric(data_merged['power'], errors='coerce')
# data_merged['power'].fillna(data_merged['power'].mean(), inplace=True)
data_merged['power'].fillna(0, inplace=True)
padded = np.append(padded, data_merged['power'].to_numpy().reshape(-1, 1), axis=1)
max_length += 1

In [21]:
data_merged['power'].to_numpy().reshape(-1, 1)

array([[0.],
       [2.],
       [3.],
       ...,
       [4.],
       [0.],
       [3.]])

In [22]:
padded[4][-2:-1]

array([0.])

### Life

In [23]:
# data_merged['life'].fillna(data_merged['life'].mean(), inplace=True)
data_merged['life'].fillna(0, inplace=True)
padded = np.append(padded, data_merged['life'].to_numpy().reshape(-1, 1), axis=1)
max_length += 1

In [24]:
data_merged['life']

1365     0.0
1367     0.0
1369     0.0
1370     0.0
1372     0.0
        ... 
42294    0.0
42295    0.0
42296    0.0
42297    0.0
42298    0.0
Name: life, Length: 20652, dtype: float64

### Preprocess prices

In [25]:
# prices.describe()
# prices.head()

In [20]:
prices_filtered = data_merged['price']

In [21]:
prices_np = prices_filtered.to_numpy()

### train, test, validation split (80, 10, 10)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(padded, prices_np, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

### Standarize

In [23]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test = y_scaler.transform(y_test.reshape(-1, 1))
y_val = y_scaler.transform(y_val.reshape(-1, 1))

## Model

In [101]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Embedding(vocab_size, 256, input_length=max_length),
# #     tf.keras.layers.LSTM(256, return_sequences=True),
#     tf.keras.layers.GRU(256, return_sequences=True),
# #     tf.keras.layers.Dense(256),
# #     tf.keras.layers.Dense(128),
# #     tf.keras.layers.Dense(64),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1)
# ])

# model.summary()

inp = tf.keras.layers.Input(shape=(max_length-5,))
emb = tf.keras.layers.Embedding(vocab_size, 256, input_length=max_length-5)(inp)
lstm = tf.keras.layers.LSTM(256, return_sequences=True)(emb)
flatten = tf.keras.layers.Flatten()(lstm)

non_text = tf.keras.layers.Input(shape=(4, ))
conc = tf.keras.layers.Concatenate()([flatten, non_text])
drop = tf.keras.layers.Dropout(0.2)(conc)
dense = tf.keras.layers.Dense(64)(drop)
output = tf.keras.layers.Dense(1)(dense)

model = Model([inp, non_text], output)

### Tensorboard

In [102]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [103]:
# optimizer = tf.keras.optimizers.RMSprop(0.0005) #todo: replace with Adam
optimizer = tf.keras.optimizers.Adam(0.00001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

In [104]:
X_train[0].shape,max_length, vocab_size

((200,), 200, 2048)

In [105]:
X_train_text, X_train_nontext = X_train[:, 0:-5], X_train[:, -5:-1]
X_val_text, X_val_nontext = X_val[:, 0:-5], X_val[:, -5:-1]

### Train

In [106]:
epochs = 500
batch_size = 64
# model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=batch_size)
# model.fit([X_train_text, X_train_nontext], y_train, epochs=epochs, callbacks=[tensorboard_callback], validation_data=(X_val, y_val), batch_size=batch_size)
model.fit([X_train_text, X_train_nontext], y_train, epochs=epochs, callbacks=[tensorboard_callback], batch_size=batch_size)

Train on 16521 samples
Epoch 1/500
16521/16521 [==============================] - 21s 1ms/sample - loss: 1.0024 - mae: 0.1338 - mse: 1.0024
Epoch 2/500
16521/16521 [==============================] - 19s 1ms/sample - loss: 0.9998 - mae: 0.1312 - mse: 0.9998
Epoch 3/500
16521/16521 [==============================] - 19s 1ms/sample - loss: 0.9975 - mae: 0.1281 - mse: 0.9975
Epoch 4/500
 6272/16521 [==========>...................] - ETA: 12s - loss: 1.5214 - mae: 0.1308 - mse: 1.5216

KeyboardInterrupt: 

### read some value

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
def decode_sentence(text):
    text = scaler.inverse_transform(text)
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

def decode_price(val):
    return y_scaler.inverse_transform(val)

In [ ]:
for i in range(100):
    price = decode_price(y_train[i])
    if price > 1:
        print(i)
        print(decode_price(y_train[i]))

In [ ]:
print(decode_sentence(X_train[70]))

In [ ]:
data_merged[data_merged['name'].str.contains("Zur the Enchanter")]

In [38]:
np.argwhere(np.isnan(padded))

array([], shape=(0, 2), dtype=int64)